In [2]:
# @title ## LD4 { display-mode: "form" }
# @markdown Įveskite savo užklausą:.


def on_button_clicked(b):
	clear_output()
	!cp -f /content/drive/MyDrive/LD4TINKA/pleaseWait.mp4 /content/
	# Display the message within the output widget.
	request = 'What is reinforcement model learning?' # @param {type:"string"}
	tokenizer = AutoTokenizer.from_pretrained(model)
	pipeline = transformers.pipeline(
		"text-generation",
		model=model,
		tokenizer=tokenizer,
		torch_dtype=torch.bfloat16,
		device_map="auto",
	)
	sequences = pipeline(
		request,
		max_length=200,
		do_sample=True,
		top_k=10,
		num_return_sequences=1,
		eos_token_id=tokenizer.eos_token_id,
	)
	from IPython.display import HTML
	from base64 import b64encode
	mp4 = open('/content/pleaseWait.mp4','rb').read()
	data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
	HTML("""
	<video controls autoplay>
	    <source src="%s" type="video/mp4">
	</video>
	""" % data_url)
	!sleep 10
	long_string = sequences[0]['generated_text']
	sentences = nltk.sent_tokenize(long_string)
	SAMPLE_RATE = 24000
	HISTORY_PROMPT = "en_speaker_1"
	chunks = ['']
	token_counter = 0
	for sentence in sentences:
		current_tokens = len(nltk.Text(sentence))
		if token_counter + current_tokens <= 250:
			token_counter = token_counter + current_tokens
			chunks[-1] = chunks[-1] + " " + sentence
		else:
			chunks.append(sentence)
			token_counter = current_tokens
	# Generate audio for each prompt
	audio_arrays = []
	for prompt in chunks:
		audio_array = generate_audio(prompt,history_prompt=HISTORY_PROMPT)
		audio_arrays.append(audio_array)
	# Combine the audio files
	combined_audio = np.concatenate(audio_arrays)
	# Write the combined audio to a file
	%cd /content/
	write_wav("combined_audio.wav", SAMPLE_RATE, combined_audio)
	%cd SadTalker
	del sentences
	del chunks
	del combined_audio
	del audio_arrays
	del sequences
	del long_string
	del token_counter
	del tokenizer
	del pipeline
	!cp -f /content/drive/MyDrive/LD4TINKA/person.png /content/
	!cp -f /content/drive/MyDrive/LD4TINKA/inference.py /content/SadTalker/inference.py
	!python3.8 /content/SadTalker/inference.py --driven_audio /content/combined_audio.wav --source_image /content/person.png --result_dir /content/video --still --preprocess full
	mp4 = open('/content/video/OutputVideo/person##combined_audio.mp4','rb').read()
	data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
	HTML("""
	<video controls autoplay>
      <source src="%s" type="video/mp4">
	</video>
	""" % data_url)

import locale
locale.getpreferredencoding = lambda: "UTF-8"

from google.colab import drive
drive.mount('/content/drive')
%cd /content/
!pip -qqq install bitsandbytes accelerate &> /dev/null
!pip install git+https://github.com/suno-ai/bark.git &> /dev/null
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader &> /dev/null
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 2 &> /dev/null
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.9 1 &> /dev/null
!sudo apt install python3.8 &> /dev/null

!sudo apt-get install python3.8-distutils &> /dev/null

!apt-get update &> /dev/null

!apt install software-properties-common &> /dev/null

!sudo dpkg --remove --force-remove-reinstreq python3-pip python3-setuptools python3-wheel &> /dev/null

!apt-get install python3-pip &> /dev/null

print('Git clone project and install requirements...')
!git clone https://github.com/Winfredy/SadTalker &> /dev/null
%cd SadTalker
!export PYTHONPATH=/content/SadTalker:$PYTHONPATH &> /dev/null
!python3.8 -m pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113 &> /dev/null
!apt update &> /dev/null
!apt install ffmpeg &> /dev/null
!python3.8 -m pip install -r requirements.txt &> /dev/null

print('Download pre-trained models...')
!rm -rf checkpoints &> /dev/null
!bash scripts/download_models.sh &> /dev/null
import ipywidgets as widgets
from IPython.display import display

#LLM modelio importai
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

#Text-to-speech importai
from bark import generate_audio,preload_models
from scipy.io.wavfile import write as write_wav
import numpy as np
import nltk
from IPython.display import Audio

model = "tiiuae/falcon-rw-1b"

nltk.download('punkt')
preload_models(
text_use_small=True,
coarse_use_small=True,
fine_use_gpu=False,
fine_use_small=True,
)

# borrow from makeittalk
import ipywidgets as widgets
import glob
import matplotlib.pyplot as plt

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

#MP4 player improtai
from IPython.display import HTML
from base64 import b64encode
from IPython.display import clear_output

button = widgets.Button(description="Generuoti atsakymą")
output = widgets.Output()

clear_output()

button.on_click(on_button_clicked)
display(button, output)